<a href="https://colab.research.google.com/github/TSGreenwood/KingCountyTransit/blob/main/ProcessingDataBusStopsRoutesZIPCodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#bus_routes_gdf = gpd.read_file('/content/drive/MyDrive/KingCountyTransit/Data/KingCountyTransitRoutes.geojson')
bus_stops_gdf = gpd.read_file('/content/drive/MyDrive/KingCountyTransit/Data/KingCountyTransitStops.geojson')
zip_codes_gdf = gpd.read_file('/content/drive/MyDrive/KingCountyTransit/Data/KingCountyZipCodes.geojson')

In [ ]:
# Check the coordinate systems
print(bus_routes_gdf.crs)
print(bus_stops_gdf.crs)
print(zip_codes_gdf.crs)

In [ ]:
print(bus_routes_gdf.info())
print(bus_stops_gdf.info())
print(zip_codes_gdf.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   ROUTE_ID            148 non-null    int64   
 1   LOCAL_EXPRESS_CODE  148 non-null    object  
 2   ROUTE_NUM           148 non-null    int64   
 3   SHAPE_Length        148 non-null    float64 
 4   geometry            148 non-null    geometry
dtypes: float64(1), geometry(1), int64(2), object(1)
memory usage: 5.9+ KB
None
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 44190 entries, 0 to 44189
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   OBJECTID                   44190 non-null  int64              
 1   CHANGE_NUM                 44190 non-null  int64              
 2   MINOR_CHANGE_NUM           44190 non-null  int64              
 

Let's clean up the bus_routes geodataframe.

In [ ]:
# All the bus routes are in service, so we'll drop some of its columns.

route_columns_to_drop = ['OBJECTID','CHANGE_NUM','MINOR_CHANGE_NUM','CURRENT_NEXT_CODE','IN_SERVICE_FLAG']
bus_routes_gdf = bus_routes_gdf.drop(columns=route_columns_to_drop)

In [ ]:
# We want all kinds of IDs to be treated as strings

route_strings = ['ROUTE_ID','ROUTE_NUM']
bus_routes_gdf[route_strings] = bus_routes_gdf[route_strings].astype(str)

In [ ]:
bus_routes_gdf.head()

In [ ]:
# Save bus_routes_gdf to a GeoJSON file in my Drive

bus_routes_path = '/content/drive/MyDrive/KingCountyTransit/Data/bus_routes.geojson'
bus_routes_gdf.to_file("bus_routes.geojson", driver='GeoJSON')

Next, we'll clean up the bus stops geodataframe. It has a lot of attributes that are not relevant. I intend to have a spatial layer of ZIP codes on top, and to engineer features such as number of bus stops in that zip code and bus stop density. Note that all of my datasets are projected to EPSG:4326 coordinate system in latitude and longitude, which does not preserve distances or areas. In order to calculate the area of a ZIP code, I'll need to reproject it to something local, like EPSG: 32610.

In [ ]:
bus_stops_gdf['STOP_STATUS'].unique()

array(['ACT', 'CLO', 'INA', 'PLN'], dtype=object)

In [ ]:
# Let's remove all entries from bus_stops_gdf where 'STOP_STATUS'   is not 'ACT'

bus_stops_gdf = bus_stops_gdf[bus_stops_gdf['STOP_STATUS'] == 'ACT']

In [ ]:
# Dropping columns from bus_stops_gdf

stops_columns_to_drop = ['OBJECTID','CHANGE_NUM','MINOR_CHANGE_NUM','ACCESSIBILITY_DECAL','AUTH_CODE','AUTH_NAME','ACCESSORY_SIGN','AWNING','BAY_CODE','BIKE_RACK',
                   'CURB','CURB_HEIGHT_FRONTDOOR','CURBPAINT','CURBPAINTLENGTH','CURRENT_NEXT_CODE','DATE_CREATED','DATE_LAST_MODIFIED','DB_MOD_DATE','EFFECTIVE_BEGINDATE',
                   'FRDOORLANDING','FRDOORSIDEWALKWIDTH','IMT_CONNECTORLIST','IN_SERVICE_FLAG','INFOSIGN_ANCHOR','KCM_MANAGED_EQUIPMENT','ROUTESIGN','ROUTESIGN_OWNER',
                   'NUM_SHELTERS','SIGN_MOUNTING_DIR','SIGNPOST','SIGNPOST_ANCHOR','SCHEDULE_HOLDER','STOP_STATUS']
bus_stops_gdf = bus_stops_gdf.drop(columns=stops_columns_to_drop)

In [ ]:
# We want all ID numbers to be strings

bus_stop_strings = ['STOP_ID','TRANS_LINK_ID','GISOBJID','ZIPCODE','FARE_ZONE']
bus_stops_gdf[bus_stop_strings] = bus_stops_gdf[bus_stop_strings].astype(str)

In [ ]:
# Create columns for the total number of bus stops in the zipcode

stops_count_per_zipcode = bus_stops_gdf.groupby('ZIPCODE').size().reset_index(name='STOPS_IN_ZIPCODE')

# Merge the count back onto the original GeoDataFrame
bus_stops_gdf = bus_stops_gdf.merge(stops_count_per_zipcode, on='ZIPCODE', how='left')

Now to process the ZIPCODE data. We're going to combine it with the bus stop data, so we need to be mindful of our geometries and projections. I want to have a visual overlay of the ZIP codes, so the ZIP code geometry should be the active one on the merged dataset.  
I want to calculate the area of each zip code. All three datasets are in the EPSG:4326 coordinate system in latitude and longitude, which does not preserve distances or areas. I'll project ZIPCODE's area onto a local projection and add it as an additional column.

In [ ]:
# All ID numbers should be strings

zip_codes_gdf['ZIPCODE'] = zip_codes_gdf['ZIPCODE'].astype(str)

In [ ]:
# Project ZIPCODE area to UTM zone 10N for calculation

zip_codes_gdf_projected = zip_codes_gdf.to_crs(epsg=32610)

In [ ]:
# Calculate area in square meters

zip_codes_gdf_projected['AREA_SQM'] = zip_codes_gdf_projected.area

In [ ]:
# Add that to the ZIP code geodataframe

zip_codes_gdf['ZIPCODE_AREA_SQM'] = zip_codes_gdf_projected['AREA_SQM']

In [ ]:
# Clarify city name

zip_codes_gdf = zip_codes_gdf.rename(columns={'PREFERRED_CITY': 'CITY_NAME'})

# Remove redundant columns

zip_columns_drop = ['OBJECTID','ZIP']
zip_codes_gdf = zip_codes_gdf.drop(columns=zip_columns_drop)

In [ ]:
zip_codes_gdf.head()

,ZIPCODE,COUNTY,ZIP_TYPE,COUNTY_NAME,CITY_NAME,Shape_Length,Shape_Area,geometry,ZIPCODE_AREA_SQM
0,98001,033,Standard,King County,AUBURN,147536.710308,5.253689e+08,"POLYGON ((-122.29032 47.35495, -122.29062 47.3...",4.876922e+07
1,98002,033,Standard,King County,AUBURN,104440.087171,2.053028e+08,"POLYGON ((-122.22820 47.35395, -122.22893 47.3...",1.905814e+07
2,98003,033,Standard,King County,FEDERAL WAY,123733.570612,3.169426e+08,"POLYGON ((-122.30392 47.35794, -122.30476 47.3...",2.942098e+07
3,98004,033,Standard,King County,BELLEVUE,99325.951735,2.507454e+08,"POLYGON ((-122.21189 47.64635, -122.21187 47.6...",2.327945e+07
4,98005,033,Standard,King County,BELLEVUE,116554.738076,2.114337e+08,"MULTIPOLYGON (((-122.15354 47.66056, -122.1535...",1.962986e+07


In [ ]:
# Save zip_codes_gdf to a GeoJSON file in my Drive

zip_codes_path = '/content/drive/MyDrive/KingCountyTransit/Data/zip_codes.geojson'
zip_codes_gdf.to_file("zip_codes.geojson", driver='GeoJSON')

Now, we'll merge that information with our bus stops data.

In [ ]:
# Merge
bus_stops_zip_codes_gdf = bus_stops_gdf.merge(zip_codes_gdf, on='ZIPCODE', how='left')

In [ ]:
bus_stops_zip_codes_gdf.head()

,STOP_ID,TRANS_LINK_ID,STOP_TYPE,TRF_DISTRICT_CODE,BEARING_CODE,CF_CROSS_STREETNAME,CF_DIST_FROM_INTERSECTION,CF_INTERSECTION_LOCCODE,DISPLACEMENT,DIST_FROM_INTERSECTION,...,geometry_x,STOPS_IN_ZIPCODE,COUNTY,ZIP_TYPE,COUNTY_NAME,CITY_NAME,Shape_Length,Shape_Area,geometry_y,ZIPCODE_AREA_SQM
0,99596,384121,REG,NW,N,N 148th St,100.0,AT,55886.0,100.0,...,POINT (-122.32435 47.73493),372,033,Standard,King County,SEATTLE,72408.042948,2.311570e+08,"POLYGON ((-122.26921 47.76943, -122.26923 47.7...",2.146118e+07
1,99597,384119,REG,NE,S,NE 130th St,100.0,AT,57269.0,100.0,...,POINT (-122.32399 47.72350),499,033,Standard,King County,SEATTLE,71062.624510,1.800824e+08,"POLYGON ((-122.32376 47.73406, -122.32382 47.7...",1.671915e+07
2,99598,384121,REG,NE,N,NE 130th St,100.0,AT,50.0,100.0,...,POINT (-122.32373 47.72350),499,033,Standard,King County,SEATTLE,71062.624510,1.800824e+08,"POLYGON ((-122.32376 47.73406, -122.32382 47.7...",1.671915e+07
3,99599,381099,LAY,NE,N,ST Light Rail,33.0,FS,33.0,33.0,...,POINT (-122.32821 47.70446),499,033,Standard,King County,SEATTLE,71062.624510,1.800824e+08,"POLYGON ((-122.32376 47.73406, -122.32382 47.7...",1.671915e+07
4,99600,381084,RL,NE,N,Pedestrian Walk,128.0,NM,2438.0,128.0,...,POINT (-122.32812 47.70301),499,033,Standard,King County,SEATTLE,71062.624510,1.800824e+08,"POLYGON ((-122.32376 47.73406, -122.32382 47.7...",1.671915e+07


In [ ]:
bus_stops_zip_codes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 23369 entries, 0 to 23368
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   STOP_ID                    23369 non-null  object  
 1   TRANS_LINK_ID              23369 non-null  object  
 2   STOP_TYPE                  23369 non-null  object  
 3   TRF_DISTRICT_CODE          23369 non-null  object  
 4   BEARING_CODE               23369 non-null  object  
 5   CF_CROSS_STREETNAME        23369 non-null  object  
 6   CF_DIST_FROM_INTERSECTION  23369 non-null  float64 
 7   CF_INTERSECTION_LOCCODE    23369 non-null  object  
 8   DISPLACEMENT               23369 non-null  float64 
 9   DIST_FROM_INTERSECTION     23369 non-null  float64 
 10  FARE_ZONE                  23369 non-null  object  
 11  HASTUS_CROSS_STREET_NAME   23369 non-null  object  
 12  INFOSIGN                   23369 non-null  object  
 13  INTERSECTION_LOC       

In [ ]:
# Calculate bus stop density by ZIPCODE

bus_stops_zip_codes_gdf['ZIPCODE_STOPS_PER_SQM'] = bus_stops_zip_codes_gdf['STOPS_IN_ZIPCODE']/bus_stops_zip_codes_gdf['ZIPCODE_AREA_SQM']

In [ ]:
bus_stops_zip_codes_gdf.head()

,STOP_ID,TRANS_LINK_ID,STOP_TYPE,TRF_DISTRICT_CODE,BEARING_CODE,CF_CROSS_STREETNAME,CF_DIST_FROM_INTERSECTION,CF_INTERSECTION_LOCCODE,DISPLACEMENT,DIST_FROM_INTERSECTION,...,STOPS_IN_ZIPCODE,COUNTY,ZIP_TYPE,COUNTY_NAME,CITY_NAME,Shape_Length,Shape_Area,geometry_y,ZIPCODE_AREA_SQM,ZIPCODE_STOPS_PER_SQM
0,99596,384121,REG,NW,N,N 148th St,100.0,AT,55886.0,100.0,...,372,033,Standard,King County,SEATTLE,72408.042948,2.311570e+08,"POLYGON ((-122.26921 47.76943, -122.26923 47.7...",2.146118e+07,0.000017
1,99597,384119,REG,NE,S,NE 130th St,100.0,AT,57269.0,100.0,...,499,033,Standard,King County,SEATTLE,71062.624510,1.800824e+08,"POLYGON ((-122.32376 47.73406, -122.32382 47.7...",1.671915e+07,0.000030
2,99598,384121,REG,NE,N,NE 130th St,100.0,AT,50.0,100.0,...,499,033,Standard,King County,SEATTLE,71062.624510,1.800824e+08,"POLYGON ((-122.32376 47.73406, -122.32382 47.7...",1.671915e+07,0.000030
3,99599,381099,LAY,NE,N,ST Light Rail,33.0,FS,33.0,33.0,...,499,033,Standard,King County,SEATTLE,71062.624510,1.800824e+08,"POLYGON ((-122.32376 47.73406, -122.32382 47.7...",1.671915e+07,0.000030
4,99600,381084,RL,NE,N,Pedestrian Walk,128.0,NM,2438.0,128.0,...,499,033,Standard,King County,SEATTLE,71062.624510,1.800824e+08,"POLYGON ((-122.32376 47.73406, -122.32382 47.7...",1.671915e+07,0.000030


In [ ]:
bus_stops_zip_codes_gdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23369 entries, 0 to 23368
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   STOP_ID                    23369 non-null  object  
 1   TRANS_LINK_ID              23369 non-null  object  
 2   STOP_TYPE                  23369 non-null  object  
 3   TRF_DISTRICT_CODE          23369 non-null  object  
 4   BEARING_CODE               23369 non-null  object  
 5   CF_CROSS_STREETNAME        23369 non-null  object  
 6   CF_DIST_FROM_INTERSECTION  23369 non-null  float64 
 7   CF_INTERSECTION_LOCCODE    23369 non-null  object  
 8   DISPLACEMENT               23369 non-null  float64 
 9   DIST_FROM_INTERSECTION     23369 non-null  float64 
 10  FARE_ZONE                  23369 non-null  object  
 11  HASTUS_CROSS_STREET_NAME   23369 non-null  object  
 12  INFOSIGN                   23369 non-null  object  
 13  INTERSECTION_LOC           2336

In [ ]:
# Add those attributes to the ZIPCODE data

zip_codes_gdf['TOTAL_BUS_STOPS'] = bus_stops_zip_codes_gdf['STOPS_IN_ZIPCODE']
zip_codes_gdf['BUS_STOPS_PER_SQM'] = zip_codes_gdf['TOTAL_BUS_STOPS']/zip_codes_gdf['ZIPCODE_AREA_SQM']

In [ ]:
# Add those attributes to the bus stop data

bus_stops_gdf['ZIPCODE_AREA_SQM'] = bus_stops_zip_codes_gdf['ZIPCODE_AREA_SQM']
bus_stops_gdf['ZIPCODE_STOPS_PER_SQM'] = bus_stops_gdf['STOPS_IN_ZIPCODE']/bus_stops_gdf['ZIPCODE_AREA_SQM']

In [ ]:
bus_stops_gdf.head()

,STOP_ID,TRANS_LINK_ID,STOP_TYPE,TRF_DISTRICT_CODE,BEARING_CODE,CF_CROSS_STREETNAME,CF_DIST_FROM_INTERSECTION,CF_INTERSECTION_LOCCODE,DISPLACEMENT,DIST_FROM_INTERSECTION,...,YCOORD,YCOORD_OFFSET,ZIPCODE,ROUTE_LIST,ON_STREET_NAME,BUS_STOP_GISOBJID,geometry,STOPS_IN_ZIPCODE,ZIPCODE_AREA_SQM,ZIPCODE_STOPS_PER_SQM
0,99596,384121,REG,NW,N,N 148th St,100.0,AT,55886.0,100.0,...,271625.6,271634.0,98155,None,ST Light Rail,100016297.0,POINT (-122.32435 47.73493),372,2.146118e+07,0.000017
1,99597,384119,REG,NE,S,NE 130th St,100.0,AT,57269.0,100.0,...,267463.4,267464.7,98125,None,ST Light Rail,100016296.0,POINT (-122.32399 47.72350),499,1.671915e+07,0.000030
2,99598,384121,REG,NE,N,NE 130th St,100.0,AT,50.0,100.0,...,267462.6,267461.3,98125,None,ST Light Rail,100016295.0,POINT (-122.32373 47.72350),499,1.671915e+07,0.000030
3,99599,381099,LAY,NE,N,ST Light Rail,33.0,FS,33.0,33.0,...,260538.8,260538.5,98125,None,ST Light Rail,100016127.0,POINT (-122.32821 47.70446),499,1.671915e+07,0.000030
4,99600,381084,RL,NE,N,Pedestrian Walk,128.0,NM,2438.0,128.0,...,260009.7,260009.4,98125,599,Northgate Sta Rail,100016114.0,POINT (-122.32812 47.70301),499,1.671915e+07,0.000030


In [ ]:
# Save the bus stops data
bus_stops_gdf.to_file('/content/drive/MyDrive/KingCountyTransit/Data/bus_stops.geojson', driver='GeoJSON')

In [ ]:
# Save the ZIPCODE data
zip_codes_gdf.to_file('/content/drive/MyDrive/KingCountyTransit/Data/zip_codes.geojson', driver='GeoJSON')